In [1]:
import numpy as np
from tic_env import TictactoeEnv, OptimalPlayer

# Tic Toc Toe environment

Our 1st game is the famous Tic Toc Toe. You can read about the game and its rules here: https://en.wikipedia.org/wiki/Tic-tac-toe

We implemented the game as an environment in the style of games in the [Python GYM library](https://gym.openai.com/). The commented source code is available in the file "tic_env.py". Here, we give a brief introduction to the environment and how it can be used.

### Initialization and attributes

You can initialize the environment / game as following:

In [2]:
env = TictactoeEnv()

Which then has the following attributes with the corresponding initial values:

In [3]:
env.__dict__

{'grid': array([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]),
 'end': False,
 'winner': None,
 'player2value': {'X': 1, 'O': -1},
 'num_step': 0,
 'current_player': 'X'}

The game is played by two players: player 'X' and player 'O'. The attribute 'current_player' shows whose turn it is. We assume that player 'X' always plays first.

The attribute 'grid' is a 3x3 numpy array and presents the board in the real game and the state $s_t$ in the reinfocement learning language. Each elements can take a value in {0, 1, -1}:
     0 : place unmarked
     1 : place marked with X 
    -1 : place marked with O 
        
The attribute 'end' shows if the game is over or not, and the attribute 'winner' shows the winner of the game: either "X", "O", or None.  

You can use function 'render' to visualize the current position of the board:

In [4]:
env.render()

|- - -|
|- - -|
|- - -|



### Taking actions

The game environment will recieve action from two players in turn and update the grid. At each time, one player can take the action $a_t$, where $a_t$ can either be an integer between 0 to 8 or a touple, corresponding to the 9 possible.

Function 'step' is used to recieve the action of the player, update the grid:

In [5]:
env.step(2)

(array([[0., 0., 1.],
        [0., 0., 0.],
        [0., 0., 0.]]),
 False,
 None)

In [6]:
env.render()

|- - X|
|- - -|
|- - -|



In [7]:
env.__dict__

{'grid': array([[0., 0., 1.],
        [0., 0., 0.],
        [0., 0., 0.]]),
 'end': False,
 'winner': None,
 'player2value': {'X': 1, 'O': -1},
 'num_step': 1,
 'current_player': 'O'}

In [8]:
env.step((1,1))

(array([[ 0.,  0.,  1.],
        [ 0., -1.,  0.],
        [ 0.,  0.,  0.]]),
 False,
 None)

In [9]:
env.render()

|- - X|
|- O -|
|- - -|



In [10]:
env.__dict__

{'grid': array([[ 0.,  0.,  1.],
        [ 0., -1.,  0.],
        [ 0.,  0.,  0.]]),
 'end': False,
 'winner': None,
 'player2value': {'X': 1, 'O': -1},
 'num_step': 2,
 'current_player': 'X'}

But not all actions are available at each time: One cannot choose a place which has been taken before. There is an error if an unavailable action is taken:

In [11]:
env.step((0,2))

ValueError: There is already a chess on position (0, 2).

### Reward

Reward is always 0 until the end of the game. When the game is over, the reward is 1 if you win the game, -1 if you lose, and 0 besides. Function 'observe' can be used after each step to recieve the new state $s_t$, whether the game is over, and the winner, and function 'reward' to get the reward value $r_t$:

In [12]:
env.observe()

(array([[ 0.,  0.,  1.],
        [ 0., -1.,  0.],
        [ 0.,  0.,  0.]]),
 False,
 None)

In [13]:
env.reward(player='X')

0

In [14]:
env.reward(player='O')

0

An example of finishing the game:

In [15]:
env.step(0)
env.step(3)
env.step(1)

(array([[ 1.,  1.,  1.],
        [-1., -1.,  0.],
        [ 0.,  0.,  0.]]),
 True,
 'X')

In [16]:
env.render()

|X X X|
|O O -|
|- - -|



In [17]:
env.observe()

(array([[ 1.,  1.,  1.],
        [-1., -1.,  0.],
        [ 0.,  0.,  0.]]),
 True,
 'X')

In [18]:
env.reward(player='X')

1

In [19]:
env.reward(player='O')

-1

# Optimal policy for Tic Toc Toe environment

Fortunately, we know the exact optimal policy for Tic Toc Toe. We have implemented and $\epsilon$-greedy version of optimal polciy which you can use for the project.

In [20]:
env.reset();

In [21]:
opt_player = OptimalPlayer(epsilon = 0., player = 'X')

In [22]:
opt_player.act(env.grid)

(0, 2)

In [23]:
opt_player.player

'X'

### An example of optimal player playing against random player

In [24]:
Turns = np.array(['X','O'])
for i in range(5):
    env.reset()
    grid, _, __ = env.observe()
    Turns = Turns[np.random.permutation(2)]
    player_opt = OptimalPlayer(epsilon=0., player=Turns[0])
    player_rnd = OptimalPlayer(epsilon=1., player=Turns[1])
    for j in range(9):
        if env.current_player == player_opt.player:
            move = player_opt.act(grid)
        else:
            move = player_rnd.act(grid)

        grid, end, winner = env.step(move, print_grid=False)

        if end:
            print('-------------------------------------------')
            print('Game end, winner is player ' + str(winner))
            print('Optimal player = ' +  Turns[0])
            print('Random player = ' +  Turns[1])
            env.render()
            env.reset()
            break


-------------------------------------------
Game end, winner is player X
Optimal player = X
Random player = O
|O - X|
|- X O|
|X - -|

-------------------------------------------
Game end, winner is player O
Optimal player = O
Random player = X
|X - O|
|X O -|
|O X -|

-------------------------------------------
Game end, winner is player O
Optimal player = O
Random player = X
|X - O|
|- O -|
|O X X|

-------------------------------------------
Game end, winner is player O
Optimal player = O
Random player = X
|X X O|
|- O X|
|O - -|

-------------------------------------------
Game end, winner is player O
Optimal player = O
Random player = X
|- X X|
|O O O|
|O X X|



### An example of optimal player playing against optimal player

In [25]:
Turns = np.array(['X','O'])
for i in range(5):
    env.reset()
    grid, _, __ = env.observe()
    Turns = Turns[np.random.permutation(2)]
    player_opt_1 = OptimalPlayer(epsilon=0., player=Turns[0])
    player_opt_2 = OptimalPlayer(epsilon=0., player=Turns[1])
    for j in range(9):
        if env.current_player == player_opt.player:
            move = player_opt_1.act(grid)
        else:
            move = player_opt_2.act(grid)

        grid, end, winner = env.step(move, print_grid=False)

        if end:
            print('-------------------------------------------')
            print('Game end, winner is player ' + str(winner))
            print('Optimal player 1 = ' +  Turns[0])
            print('Optimal player 2 = ' +  Turns[1])
            env.render()
            env.reset()
            break


-------------------------------------------
Game end, winner is player None
Optimal player 1 = O
Optimal player 2 = X
|X O O|
|O X X|
|X X O|

-------------------------------------------
Game end, winner is player None
Optimal player 1 = O
Optimal player 2 = X
|X O X|
|X O O|
|O X X|

-------------------------------------------
Game end, winner is player None
Optimal player 1 = X
Optimal player 2 = O
|X X O|
|O O X|
|X O X|

-------------------------------------------
Game end, winner is player None
Optimal player 1 = O
Optimal player 2 = X
|X O X|
|O O X|
|X X O|

-------------------------------------------
Game end, winner is player None
Optimal player 1 = X
Optimal player 2 = O
|X X O|
|O O X|
|X O X|



In [1]:
from utils import play
from qlearner import QPlayer
from tic_env import OptimalPlayer

optimal_player = OptimalPlayer(epsilon=1.0)
q_player = QPlayer()

play(optimal_player, q_player, episodes=10)

Current qstate:
|- - -|
|- - -|
|X - -|

Next action: (0, 0)
Simulation: end=False, reward=0
Current qvalues:
defaultdict(<class 'int'>, {})
Current qstate:
|O - -|
|- X -|
|X - -|

Next action: (0, 1)
Simulation: end=False, reward=0
Current qvalues:
defaultdict(<class 'int'>, {state=(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0), action=(0, 0): 0.0, state=(-1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0), action=(0, 1): 0})
Current qstate:
|O O -|
|- X -|
|X X -|

Next action: (1, 0)
Simulation: end=False, reward=0
Current qvalues:
defaultdict(<class 'int'>, {state=(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0), action=(0, 0): 0.0, state=(-1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0), action=(0, 1): 0.0, state=(-1.0, -1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0), action=(0, 2): 0})
Current qstate:
|- - -|
|- - -|
|- - -|

Next action: (0, 1)
Simulation: end=False, reward=0
Current qvalues:
defaultdict(<class 'int'>, {state=(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0), action=(0, 0): 0.0, state

({'wins': 2, 'losses': 3, 'M': -0.2}, {'wins': 3, 'losses': 2, 'M': 0.2})

In [3]:
pos_qvalues = [(qstate, qvalue) for qstate, qvalue in q_player.qvalues.items() if qvalue > 0]

In [4]:
pos_qvalues

[(state=(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), action=(0, 0),
  0.21698493414362285),
 (state=(-1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0), action=(0, 2),
  0.24207957607437766),
 (state=(-1.0, 1.0, -1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0), action=(1, 0),
  0.02216817767024824),
 (state=(-1.0, 1.0, -1.0, -1.0, 1.0, 1.0, 0.0, 0.0, 1.0), action=(2, 0),
  0.9661344643619678),
 (state=(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0), action=(0, 0),
  0.39880529732792847),
 (state=(-1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0), action=(0, 1),
  0.13935001547278444),
 (state=(-1.0, 0.0, -1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0), action=(0, 1),
  0.9404614448944706),
 (state=(-1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0), action=(0, 1),
  0.5343594515610484),
 (state=(-1.0, -1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0), action=(0, 2),
  0.9709645363823421),
 (state=(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), action=(0, 0),
  0.5450761625295887),
 (state=(1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 0.0

In [7]:
neg_qvalues = [(qstate, qvalue) for qstate, qvalue in q_player.qvalues.items() if qvalue < 0]

In [8]:
neg_qvalues

[(state=(1.0, 1.0, -1.0, -1.0, 1.0, 1.0, 0.0, -1.0, -1.0), action=(2, 0),
  -0.8339166160123925),
 (state=(1.0, -1.0, 1.0, 1.0, -1.0, -1.0, -1.0, 1.0, 0.0), action=(2, 2),
  -0.6764664550262907),
 (state=(1.0, 1.0, -1.0, -1.0, -1.0, 1.0, 1.0, -1.0, 0.0), action=(2, 2),
  -0.7364799055342576),
 (state=(1.0, -1.0, 1.0, 1.0, -1.0, -1.0, -1.0, 0.0, 1.0), action=(2, 1),
  -0.8576042586536251),
 (state=(-1.0, 1.0, 1.0, 1.0, 1.0, -1.0, -1.0, -1.0, 0.0), action=(2, 2),
  -0.26490810937500003),
 (state=(-1.0, 1.0, 1.0, 1.0, -1.0, -1.0, 1.0, -1.0, 0.0), action=(2, 2),
  -0.05),
 (state=(1.0, 1.0, -1.0, -1.0, -1.0, 0.0, 1.0, -1.0, 1.0), action=(1, 2),
  -0.7080109756612273),
 (state=(1.0, -1.0, 1.0, -1.0, -1.0, 1.0, 1.0, 0.0, -1.0), action=(2, 1),
  -0.45963991233736307),
 (state=(1.0, -1.0, 1.0, 1.0, -1.0, 1.0, -1.0, 0.0, -1.0), action=(2, 1),
  -0.7853612360570624),
 (state=(1.0, -1.0, 1.0, 1.0, 1.0, -1.0, -1.0, 0.0, -1.0), action=(2, 1),
  -0.985840130886649),
 (state=(1.0, -1.0, 1.0, -1.0, 1.